In [3]:
#exp no:6
!pip install --upgrade typing_extensions
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.8.0+cpu
CUDA available: False


In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

texts = ["I really enjoyed the movie!",
         "The book was boring.",
         "The restaurant had amazing food.",
         "The service was terrible."]
labels = [1, 0, 1, 0]

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

input_ids, attention_masks = tokenized["input_ids"], tokenized["attention_mask"]
labels = torch.tensor(labels)

train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels = train_test_split(
    input_ids, attention_masks, labels, random_state=42, test_size=0.2
)

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        b_in, b_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_in, attention_mask=b_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch in test_loader:
        b_in, b_mask, b_labels = batch
        logits = model(b_in, attention_mask=b_mask).logits
        preds.extend(torch.argmax(logits, dim=1).tolist())
        trues.extend(b_labels.tolist())

print(f"Accuracy: {accuracy_score(trues, preds):.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.8147
Epoch 2, Loss: 0.6380
Epoch 3, Loss: 0.5517
Accuracy: 0.0000
